<a href="https://colab.research.google.com/github/bomiiisong/AI_Project/blob/master/Modeling/VGG_final_1892.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import keras
import tensorflow

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!ls -l '/content/drive/My Drive/AI_team3/img_final_mdx.zip'

-rw------- 1 root root 311857006 Apr 20 06:29 '/content/drive/My Drive/AI_team3/img_final_mdx.zip'


In [ ]:
!unzip /content/drive/My\ Drive/AI_team3/img_final_mdx.zip

In [8]:
image_dir = 'img'

In [9]:
from keras.preprocessing.image import ImageDataGenerator

random_seed = 414

datagen = ImageDataGenerator(rescale = 1./255, validation_split=0.3)

train_generator = datagen.flow_from_directory(image_dir,target_size = (640, 640), seed=random_seed,
                                                    batch_size = 25, class_mode = 'binary', subset='training')
valid_generator = datagen.flow_from_directory(image_dir, target_size=(640, 640), seed=random_seed,
                                              batch_size=21, class_mode='binary', subset='validation')

Found 1325 images belonging to 2 classes.
Found 567 images belonging to 2 classes.


In [ ]:
for pred, label in train_generator:
  print(pred.shape)
  print(label.shape)
  print(label)
  break

In [10]:
from keras.applications import VGG16

conv_base = VGG16(weights = 'imagenet', include_top = False, input_shape = (640, 640, 3))

set_trainable = False

for layer in conv_base.layers:
  if layer.name == 'block5_conv1':       
    set_trainable = True

  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

conv_base.summary()

58892288/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 640, 640, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 640, 640, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 640, 640, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 320, 320, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 320, 320, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 320, 320, 128)     147584    
______________________________________________________________

In [11]:
from keras import models, layers, optimizers

model = models.Sequential()
model.add(conv_base)

model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation = 'sigmoid'))   

model.summary()


model.compile(loss = 'binary_crossentropy', 
              optimizer = optimizers.Adam(lr = 0.000005),
              metrics = ['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 20, 20, 512)       14714688  
_________________________________________________________________
flatten (Flatten)            (None, 204800)            0         
_________________________________________________________________
dense (Dense)                (None, 256)               52429056  
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 67,144,001
Trainable params: 59,508,737
Non-trainable params: 7,635,264
_________________________________________________________________


In [12]:
%%time

Hist = model.fit(train_generator, 
                 steps_per_epoch = train_generator.samples // train_generator.batch_size, 
                 epochs = 50, 
                 validation_data = valid_generator,
                 validation_steps=valid_generator.samples // valid_generator.batch_size)

Epoch 1/50
53/53 [==============================] - 162s 2s/step - loss: 0.6958 - accuracy: 0.5970 - val_loss: 0.4599 - val_accuracy: 0.7937
Epoch 2/50
53/53 [==============================] - 85s 2s/step - loss: 0.3469 - accuracy: 0.8639 - val_loss: 0.3550 - val_accuracy: 0.8201
Epoch 3/50
53/53 [==============================] - 85s 2s/step - loss: 0.2013 - accuracy: 0.9292 - val_loss: 0.3134 - val_accuracy: 0.8501
Epoch 4/50
53/53 [==============================] - 85s 2s/step - loss: 0.1428 - accuracy: 0.9536 - val_loss: 0.3081 - val_accuracy: 0.8624
Epoch 5/50
53/53 [==============================] - 85s 2s/step - loss: 0.1134 - accuracy: 0.9685 - val_loss: 0.2785 - val_accuracy: 0.8765
Epoch 6/50
53/53 [==============================] - 85s 2s/step - loss: 0.0582 - accuracy: 0.9893 - val_loss: 0.2803 - val_accuracy: 0.8748
Epoch 7/50
53/53 [==============================] - 86s 2s/step - loss: 0.0395 - accuracy: 0.9939 - val_loss: 0.3147 - val_accuracy: 0.8624
Epoch 8/50
53/53 [=

In [13]:
loss, accuracy = model.evaluate(valid_generator, steps=valid_generator.samples // valid_generator.batch_size)
loss, accuracy

27/27 [==============================] - 26s 957ms/step - loss: 0.4946 - accuracy: 0.8871


(0.49464625120162964, 0.8871251940727234)

In [14]:
model.save('/content/drive/My Drive/Colab Notebooks/models/VGG16_1892.h5') # 1시간 12분 29초 소요

In [15]:
!ls -l /content/drive/My\ Drive/Colab\ Notebooks/models

total 3745504
-rw------- 1 root root      34592 Mar 17 07:31 001_Model_iris.h5
-rw------- 1 root root   41499744 Mar 23 06:47 002_dogs_and_cats_Augmentation.h5
-rw------- 1 root root   41498896 Mar 23 02:16 002_dogs_and_cats_small.h5
-rw------- 1 root root   25199032 Mar 24 05:15 004_dogs_and_cats_feature_extraction.h5
-rw------- 1 root root  140748400 Mar 24 08:15 005_dogs_and_cats_fine_tuning.h5
-rw------- 1 root root   41661328 Mar 23 06:45 201_CNN_Logo_Augmentation.h5
-rw------- 1 root root 2800023136 Apr 20 08:36 ResNet_all_train.h5
-rw------- 1 root root  744728176 Apr 20 10:34 VGG16_1892.h5


In [17]:
from keras.models import load_model
model_loading = load_model('/content/drive/My Drive/Colab Notebooks/models/VGG16_1892.h5')

In [18]:
loss, accuracy = model_loading.evaluate(valid_generator, steps=valid_generator.n // valid_generator.batch_size)

print(f'Loss : {loss:.5f}')
print(f'Accuracy : {accuracy:.5f}')

27/27 [==============================] - 25s 922ms/step - loss: 0.4946 - accuracy: 0.8871
Loss : 0.49465
Accuracy : 0.88713


## Visualization

In [19]:
import pathlib
from glob import glob

data_root = pathlib.Path(glob('/content/*/*/')[0])
print(data_root)
# label = ['0','1']

/content/img/1


In [ ]:
import random

all_image_paths = list(data_root.glob('*'))
all_image_paths = [str(path) for path in all_image_paths]

# random.choice(all_image_paths)

all_image_paths

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))

paths = all_image_paths[:10]

for idx, image in enumerate(paths) :
    img = cv2.imread(image)
    img = cv2.resize(img, dsize=(640,640))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)

    logits = model_loading.predict(img)[0]
    
    plt.subplot(2, 5, idx+1)
    plt.imshow(plt.imread(image))
    label = image.split('/')[-2]
    plt.title("label : " + str(label))
    plt.xlabel(logits)

* worst

In [23]:
import pathlib
from glob import glob

data_root_worst = pathlib.Path(glob('/content/*/*/')[1])
print(data_root_worst)
# label = ['0','1']

/content/img/0


In [ ]:
import random

worst_image_paths = list(data_root_worst.glob('*'))
worst_image_paths = [str(path) for path in worst_image_paths]

# random.choice(all_image_paths)

worst_image_paths

In [ ]:
import cv2
import numpy as np

plt.figure(figsize=(20,10))

paths_worst = worst_image_paths[:10]

for idx, image in enumerate(paths_worst) :
    worst_img = cv2.imread(image)
    worst_img = cv2.resize(worst_img, dsize=(640,640))
    worst_img = worst_img / 255.0
    worst_img = np.expand_dims(worst_img, axis=0)

    logits_2 = model_loading.predict(worst_img)[0]
    
    plt.subplot(2, 5, idx+1)
    plt.imshow(plt.imread(image))
    label = image.split('/')[-2]
    plt.title("label : " + str(label))
    plt.xlabel(logits_2)